In [1]:
import tensorflow as tf

In [3]:
model = tf.keras.models.load_model('InceptionV3.keras')

In [4]:
import pickle
with open("calibration_InceptionV3.pkl", "rb") as file:
  calibration = pickle.load(file)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
!pip install pydicom

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.4 MB/s eta 0:00:00


In [6]:
!mkdir lib

In [7]:
!mkdir lib/somelib

In [8]:
import os
import shutil
shutil.move('0004cfab-14fd-4e49-80ba-63a80b6bddd6.dcm', 'lib/somelib/0004cfab-14fd-4e49-80ba-63a80b6bddd6.dcm')
shutil.move('000924cf-0f8d-42bd-9158-1af53881a557.dcm', 'lib/somelib//000924cf-0f8d-42bd-9158-1af53881a557.dcm')

'lib/somelib//000924cf-0f8d-42bd-9158-1af53881a557.dcm'

In [9]:
from tqdm import tqdm
import pydicom
from PIL import Image
import os
import shutil

def convert_dicom_to_jpeg(input_folder, output_folder):
    for filename in tqdm(os.listdir(input_folder)):
        if filename.endswith('.dcm'):
            dicom_file = os.path.join(input_folder, filename)
            dicom_data = pydicom.dcmread(dicom_file)

            image_array = dicom_data.pixel_array
            jpeg_file = os.path.splitext(filename)[0] + '.jpeg'
            jpeg_path = os.path.join(output_folder, jpeg_file)
            image = Image.fromarray(image_array)
            image.save(jpeg_path, "JPEG")
convert_dicom_to_jpeg('lib/somelib', 'lib/somelib')


100%|██████████| 2/2 [00:00<00:00, 57.53it/s]


In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
test_datagen = ImageDataGenerator()#rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        'lib',
        target_size=(256, 256),
        batch_size=2,
        class_mode='binary',
        shuffle=False)


Found 2 images belonging to 1 classes.


In [11]:
def predict(photos) :
    predictions=model.predict(photos)
    calibrated_probabilities=calibration.predict_proba(predictions)[:, 1]
    return calibrated_probabilities

In [12]:
pred=predict(test_generator)

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step


In [13]:
print(pred)

[0.0957279  0.14263269]


In [14]:
test_generator.filenames

['somelib/0004cfab-14fd-4e49-80ba-63a80b6bddd6.jpeg',
 'somelib/000924cf-0f8d-42bd-9158-1af53881a557.jpeg']